In [1]:
"""
코랩용 DPO (Direct Preference Optimization) DataSet 생성 스크립트
./random_persona_campaign.csv 파일을 토대로 각 입력 데이터에 맞는 CRM 예시를 생성해
/content/drive/MyDrive/멋사/dpo_dataset/에 저장
"""

'\n코랩용 DPO (Direct Preference Optimization) DataSet 생성 스크립트\n./random_persona_campaign.csv 파일을 토대로 각 입력 데이터에 맞는 CRM 예시를 생성해\n/content/drive/MyDrive/멋사/dpo_dataset/에 저장\n'

In [1]:
import torch
torch.cuda.is_available()

# from google.colab import drive
# drive.mount('/content/drive')

import os
print(os.getcwd())
print(os.listdir())

/content
['.config', '.env', 'drive', '.ipynb_checkpoints', 'AmoRe_crm_generator', 'sample_data']


In [ ]:
!pip install datasets peft trl bitsandbytes accelerate
!pip install -U transformers
!pip show transformers

In [3]:
!git clone https://github.com/jjjh02/AmoRe_crm_generator.git
%cd AmoRe_crm_generator
!git checkout jinhyeok

Cloning into 'AmoRe_crm_generator'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 108 (delta 46), reused 83 (delta 30), pack-reused 0 (from 0)
Receiving objects: 100% (108/108), 1.86 MiB | 12.30 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/AmoRe_crm_generator
Branch 'jinhyeok' set up to track remote branch 'jinhyeok' from 'origin'.
Switched to a new branch 'jinhyeok'


In [3]:
# !git branch
os.chdir("/content/AmoRe_crm_generator/finetuning")
print(os.getcwd())

/content/AmoRe_crm_generator/finetuning


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import argparse
import csv
import inspect
import json
import os
import re
import sys
import tempfile
import urllib.error
import urllib.request
from contextlib import redirect_stdout
from io import StringIO


BASE_DIR = os.getcwd()
DEFAULT_CSV = os.path.join(BASE_DIR, "random_persona_campaign.csv")
# DEFAULT_OUTPUT = os.path.join(BASE_DIR, "finetuning_data_dpo", "cycle_01_v2.json")
DEFAULT_OUTPUT = "/content/drive/MyDrive/멋사/dataset_dpo/cycle_01_v2.json"
ADAPTER_PATH = "/content/drive/MyDrive/멋사/adapters_dpo/adapters_dpo_2"
_ADAPTER_PATCHED = True    # 어댑터 패치 완료 여부
SRC_DIR = os.path.abspath(os.path.join(BASE_DIR, "..", "src"))

PIPELINE_MODULE = None
PIPELINE_CONTEXT = None
PIPELINE_DEFAULTS = {
    "qwen_model": "Qwen/Qwen2.5-1.5B-Instruct",
    "exa_model": "LGAI-EXAONE/EXAONE-4.0-1.2B",
    "top_k": 3,
}

ENABLE_PIPELINE_CACHE = True
ENABLE_BATCH = True

TIMING_AVG_WINDOW = 10
_TIMING_SUM = {
    "load": 0.0,
    "qwen": 0.0,
    "rag": 0.0,
    "exaone": 0.0,
    "total": 0.0,
}
_TIMING_COUNT = 0

def _log_timing(timing):
    global _TIMING_COUNT
    _TIMING_COUNT += 1
    for key in _TIMING_SUM:
        _TIMING_SUM[key] += float(timing.get(key, 0.0))
    _log(
        "[Timing] "
        f"load={timing.get('load', 0.0):.2f}s "
        f"qwen={timing.get('qwen', 0.0):.2f}s "
        f"rag={timing.get('rag', 0.0):.2f}s "
        f"exaone={timing.get('exaone', 0.0):.2f}s "
        f"total={timing.get('total', 0.0):.2f}s"
    )
    if _TIMING_COUNT % TIMING_AVG_WINDOW == 0:
        avg = {k: _TIMING_SUM[k] / TIMING_AVG_WINDOW for k in _TIMING_SUM}
        _log(
            "[TimingAvg] "
            f"n={TIMING_AVG_WINDOW} "
            f"load={avg['load']:.2f}s "
            f"qwen={avg['qwen']:.2f}s "
            f"rag={avg['rag']:.2f}s "
            f"exaone={avg['exaone']:.2f}s "
            f"total={avg['total']:.2f}s"
        )
        for key in _TIMING_SUM:
            _TIMING_SUM[key] = 0.0

def _log(message):
    print(message)


def _import_pipeline_main():
    global PIPELINE_MODULE
    if SRC_DIR not in sys.path:
        sys.path.insert(0, SRC_DIR)
    try:
        import run_qwen_exaone_pipeline as pipeline_module
        pipeline_main = pipeline_module.main
    except Exception as exc:
        raise ImportError(
            "Failed to import main from ../src/run_qwen_exaone_pipeline.py"
        ) from exc
    _apply_exaone_adapter()
    PIPELINE_MODULE = pipeline_module
    return pipeline_main


def _get_pipeline_context():
    global PIPELINE_CONTEXT
    if PIPELINE_CONTEXT is not None:
        return PIPELINE_CONTEXT
    if PIPELINE_MODULE is None:
        _import_pipeline_main()
    if PIPELINE_MODULE is not None:
        if hasattr(PIPELINE_MODULE, "_set_cache_enabled"):
            PIPELINE_MODULE._set_cache_enabled(ENABLE_PIPELINE_CACHE)
        else:
            try:
                PIPELINE_MODULE.CACHE_ENABLED = ENABLE_PIPELINE_CACHE
            except Exception:
                pass
    base = os.path.dirname(os.path.dirname(os.path.abspath(PIPELINE_MODULE.__file__)))
    if not ENABLE_PIPELINE_CACHE:
        return {
            "base": base,
            "data": None,
            "q_generator": None,
            "exa_generator": None,
        }
    data = None
    q_generator = None
    exa_generator = None
    try:
        data = PIPELINE_MODULE._load_data(base)
    except Exception:
        data = None
    try:
        q_generator = PIPELINE_MODULE._get_qwen_generator(PIPELINE_DEFAULTS["qwen_model"])
    except Exception:
        q_generator = None
    try:
        exa_generator = PIPELINE_MODULE._get_exaone_generator(PIPELINE_DEFAULTS["exa_model"])
    except Exception:
        exa_generator = None
    PIPELINE_CONTEXT = {
        "base": base,
        "data": data,
        "q_generator": q_generator,
        "exa_generator": exa_generator,
    }
    return PIPELINE_CONTEXT


def _apply_exaone_adapter():
    global _ADAPTER_PATCHED
    if _ADAPTER_PATCHED:
        return
    if not ADAPTER_PATH:
        return
    if not os.path.exists(ADAPTER_PATH):
        raise FileNotFoundError(f"Adapter not found: {ADAPTER_PATH}")

    import run_qwen_exaone_pipeline as pipeline_module
    import tone_correction
    try:
        from peft import PeftModel
    except ImportError as exc:
        raise RuntimeError("peft is required to load adapters.") from exc

    class PatchedExaoneToneCorrector(tone_correction.ExaoneToneCorrector):
        _cache = {}

        def __init__(self, model_name="LGAI-EXAONE/EXAONE-4.0-1.2B", use_cache=True):
            cache_allowed = use_cache and ENABLE_PIPELINE_CACHE
            key = (model_name, ADAPTER_PATH)
            cached = self._cache.get(key) if cache_allowed else None
            if cached:
                self.device = cached["device"]
                self.model_name = model_name
                self.tokenizer = cached["tokenizer"]
                self.model = cached["model"]
                return
            super().__init__(model_name=model_name, use_cache=use_cache)
            self.model = PeftModel.from_pretrained(self.model, ADAPTER_PATH)
            try:
                self.model.eval()
            except Exception:
                pass
            if cache_allowed:
                self._cache[key] = {
                    "device": self.device,
                    "tokenizer": self.tokenizer,
                    "model": self.model,
                }

    pipeline_module.ExaoneToneCorrector = PatchedExaoneToneCorrector
    _ADAPTER_PATCHED = True
    _log(f"[Adapter] Loaded: {ADAPTER_PATH}")

def _parse_bool(value):
    if isinstance(value, bool):
        return value
    if value is None:
        return False
    if isinstance(value, (int, float)):
        return bool(value)
    text = str(value).strip().lower()
    return text in {"1", "true", "yes", "y", "t"}


def _load_pairs(csv_path):
    with open(csv_path, "r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            if not row:
                continue
            persona_raw = row.get("persona", "").strip()
            brand_raw = row.get("brand", "").strip()
            product_raw = row.get("product", "").strip()
            stage_raw = row.get("stage_index", "").strip()
            style_raw = row.get("style_index", "").strip()
            if not persona_raw or not brand_raw or not product_raw:
                continue
            if not stage_raw or not style_raw:
                continue
            try:
                persona = int(persona_raw)
                stage_index = int(stage_raw)
                style_index = int(style_raw)
            except ValueError:
                continue
            yield {
                "persona": persona,
                "brand": brand_raw,
                "product": product_raw,
                "stage_index": stage_index,
                "style_index": style_index,
                "is_event": _parse_bool(row.get("is_event", "")),
            }


def _format_prompt(summarization):
    if summarization is None:
        return ""
    if isinstance(summarization, str):
        return summarization.strip()
    return json.dumps(summarization, ensure_ascii=False, indent=2)


def _format_event(selected_event):
    if selected_event in (None, "", {}):
        return "없음"
    if isinstance(selected_event, dict):
        for key in ("title", "name", "event_name", "event"):
            if selected_event.get(key):
                return str(selected_event.get(key))
        return json.dumps(selected_event, ensure_ascii=False)
    return str(selected_event)


def _format_price(price):
    if price in (None, ""):
        return ""
    if isinstance(price, (int, float)):
        return f"{int(price):,}원"
    text = str(price).strip()
    if not text:
        return ""
    if "원" in text:
        return text
    if text.replace(",", "").isdigit():
        return f"{int(text.replace(',', '')):,}원"
    return text


def _format_persona(persona_profile):
    if not isinstance(persona_profile, dict):
        return str(persona_profile or "")
    name = persona_profile.get("name", "")
    extras = []
    value_focus = persona_profile.get("value_focus")
    skin_type = persona_profile.get("skin_type")
    traits = persona_profile.get("traits")
    shopping_style = persona_profile.get("shopping_style")
    if value_focus:
        extras.append(str(value_focus))
    if skin_type:
        extras.append(str(skin_type))
    if traits:
        if isinstance(traits, list):
            extras.append(", ".join([str(t) for t in traits if t]))
        else:
            extras.append(str(traits))
    if shopping_style:
        extras.append(str(shopping_style))
    extra_text = ", ".join([e for e in extras if e])
    if name and extra_text:
        return f"{name} ({extra_text})"
    return name or extra_text


def _build_prompt_text(meta, fallback_text):
    persona = _format_persona(meta.get("persona_profile") if isinstance(meta, dict) else None)
    stage = ""
    if isinstance(meta, dict):
        stage = meta.get("stage_name") or meta.get("stage_kr") or ""
    brand = meta.get("brand") if isinstance(meta, dict) else ""
    product_basic = meta.get("product_basic") if isinstance(meta, dict) else None
    product_name = ""
    price = ""
    if isinstance(product_basic, dict):
        product_name = product_basic.get("name", "") or ""
        price = _format_price(product_basic.get("price"))
    product_query = meta.get("product_query") if isinstance(meta, dict) else ""
    if not product_name:
        product_name = product_query or ""

    event_text = _format_event(meta.get("selected_event") if isinstance(meta, dict) else None)

    lines = ["[컨텍스트]"]
    if persona:
        lines.append(f"- Persona: {persona}")
    if stage:
        lines.append(f"- Stage: {stage}")
    if brand or product_name:
        lines.append(f"- Brand/Product: {brand} / {product_name}".strip())
    if price:
        lines.append(f"- Price: {price}")
    lines.append(f"- Event: {event_text}")

    prompt = "\n".join(lines).strip()
    if prompt:
        return prompt
    return _format_prompt(fallback_text)


def _candidate_text(candidate):
    if isinstance(candidate, dict):
        return (
            candidate.get("text")
            or candidate.get("crm_message")
            or candidate.get("message")
            or candidate.get("content")
        )
    return str(candidate)


def _candidate_meta(candidate):
    if isinstance(candidate, dict):
        return candidate.get("meta") or {}
    return {}


def _normalize_candidates(raw):
    if raw is None:
        return []
    if isinstance(raw, dict):
        if "candidates" in raw:
            raw = raw["candidates"]
        elif "messages" in raw:
            raw = raw["messages"]
        elif "crm_messages" in raw:
            raw = raw["crm_messages"]
        elif "crm_message" in raw and isinstance(raw["crm_message"], list):
            raw = raw["crm_message"]
        else:
            raw = [raw]
    if not isinstance(raw, list):
        raw = [raw]

    normalized = []
    for idx, item in enumerate(raw):
        text = _candidate_text(item)
        if not text:
            continue
        normalized.append(
            {
                "response_id": idx,
                "text": text,
                "meta": _candidate_meta(item),
            }
        )
    return normalized


def _dedupe_candidates(items):
    seen = set()
    result = []
    for item in items:
        text = _candidate_text(item)
        if not text:
            continue
        key = text.strip()
        if not key or key in seen:
            continue
        seen.add(key)
        result.append(item)
    return result


def _extract_response_text(data):
    if isinstance(data, dict):
        output_text = data.get("output_text")
        if isinstance(output_text, str) and output_text.strip():
            return output_text.strip()

        output = data.get("output")
        if isinstance(output, list):
            parts = []
            for item in output:
                if not isinstance(item, dict):
                    continue
                content = item.get("content", [])
                if isinstance(content, list):
                    for block in content:
                        if isinstance(block, dict) and isinstance(block.get("text"), str):
                            parts.append(block["text"])
                        elif isinstance(block, str):
                            parts.append(block)
                elif isinstance(content, str):
                    parts.append(content)
            if parts:
                return "".join(parts).strip()

    raise ValueError(f"Invalid evaluator response: {data}")


def _format_meta(meta):
    if not isinstance(meta, dict):
        return str(meta)

    lines = []
    persona_profile = meta.get("persona_profile")
    if persona_profile is not None:
        lines.append(f"persona_profile: {json.dumps(persona_profile, ensure_ascii=False)}")
    brand = meta.get("brand")
    if brand:
        lines.append(f"brand: {brand}")
    stage_kr = meta.get("stage_kr")
    if stage_kr:
        lines.append(f"stage_kr: {stage_kr}")
    objective = meta.get("objective")
    if objective:
        lines.append(f"objective: {objective}")
    target_state = meta.get("target_state")
    if target_state:
        lines.append(f"target_state: {target_state}")
    style_templates = meta.get("style_templates")
    if style_templates:
        if isinstance(style_templates, list):
            lines.append("style_templates:")
            for item in style_templates:
                lines.append(f"- {item}")
        else:
            lines.append(f"style_templates: {style_templates}")
    selected_event = meta.get("selected_event")
    if selected_event is not None:
        if isinstance(selected_event, (dict, list)):
            event_text = json.dumps(selected_event, ensure_ascii=False)
        else:
            event_text = str(selected_event)
        lines.append(f"selected_event: {event_text}")

    return "\n".join(lines) if lines else "(context unavailable)"


def _call_gpt(prompt_text, candidates):
    api_key = os.environ.get("OPENAI_API_KEY")
    if not api_key:
        raise RuntimeError("OPENAI_API_KEY is not set.")

    candidate_lines = []
    for idx, candidate in enumerate(candidates):
        text = candidate.get("text", "")
        candidate_lines.append(f"[{idx}]\n{text}")
    candidate_block = "\n\n".join(candidate_lines)

    system_prompt = (
        "너는 CRM 메시지의 불필요한 글자/표현을 평가하는 심사자다.\n"
        "목표는 가장 깔끔하고 불필요한 요소가 적은 메시지를 고르는 것이다.\n"
        "\n"
        "불필요한 요소 예시:\n"
        "- JSON/코드 조각(예: ```json, { } ) 또는 [제목] 같은 괄호 표기\n"
        "- 영어/한국어의 어색한 혼용(브랜드/제품 고유명 제외, 나머지는 한국어만 허용)\n"
        "- 컨텍스트의 페르소나/개인정보를 그대로 호명(예: Budget Seeker 님)\n"
        "- 과도한 특수문자/이모지/구분선\n"
        "\n"
        "의미 전달에 필요한 핵심 문장만 남긴 후보를 선택하라.\n"
        "출력은 반드시 JSON 하나로만 해라. 형식: {\"best_index\":0,\"reason_best\":\"...\",\"reason_others\":[{\"index\":1,\"reason\":\"...\"}]}\n"
        "reason은 1~2문장으로 간단하게 작성하라.\n"
    )
    user_prompt = (
        "컨텍스트\n"
        f"{prompt_text}\n\n"
        "후보\n"
        f"{candidate_block}\n\n"
        "JSON만 출력하라."
    )

    payload = {
        "model": "gpt-5-nano",
        "input": [
            {
                "role": "system",
                "content": [{"type": "input_text", "text": system_prompt}],
            },
            {
                "role": "user",
                "content": [{"type": "input_text", "text": user_prompt}],
            },
        ],
    }

    request = urllib.request.Request(
        "https://api.openai.com/v1/responses",
        data=json.dumps(payload).encode("utf-8"),
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        },
        method="POST",
    )

    try:
        with urllib.request.urlopen(request, timeout=30) as response:
            data = json.loads(response.read().decode("utf-8"))
    except urllib.error.HTTPError as exc:
        body = exc.read().decode("utf-8", errors="replace")
        raise RuntimeError(f"OpenAI API error {exc.code}: {body}") from exc

    content = _extract_response_text(data)
    text = content.strip()
    if text.startswith("```"):
        text = re.sub(r"^```[a-zA-Z0-9]*", "", text)
        text = re.sub(r"```$", "", text).strip()

    parsed = None
    match_json = re.search(r"\{.*\}", text, re.DOTALL)
    if match_json:
        try:
            parsed = json.loads(match_json.group(0))
        except json.JSONDecodeError:
            parsed = None

    best_index = None
    reason_best = ""
    reason_others = []
    if isinstance(parsed, dict):
        best_index = parsed.get("best_index")
        if best_index is None:
            best_index = parsed.get("index") or parsed.get("choice")
        reason_best = parsed.get("reason_best") or parsed.get("reason") or ""
        reason_others = parsed.get("reason_others") or parsed.get("reasons_others") or []
    if best_index is None:
        match = re.search(r"-?\d+", text)
        if not match:
            raise ValueError(f"Invalid evaluator response: {content}")
        best_index = int(match.group(0))
    if isinstance(best_index, str) and best_index.strip().isdigit():
        best_index = int(best_index.strip())
    if best_index is None or best_index < 0 or best_index >= len(candidates):
        raise ValueError(f"Evaluator index out of range: {best_index}")
    if isinstance(reason_others, dict):
        reason_others = [
            {"index": k, "reason": v} for k, v in reason_others.items()
        ]
    return best_index, reason_best, reason_others

def _load_existing_records(path):
    if not os.path.exists(path):
        return []
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    if not isinstance(data, list):
        raise ValueError(f"Expected a list in {path}")
    return data


def _save_records(path, records):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(records, f, ensure_ascii=False, indent=2)


def _extract_pipeline_output(result):
    if isinstance(result, dict):
        if "qwen" in result or "exaone" in result:
            qwen = result.get("qwen", {}) or {}
            exaone = result.get("exaone", {}) or {}
            summarization = qwen.get("draft") or qwen.get("qwen_draft")
            crm_message = exaone.get("result_raw") or exaone.get("crm_message")
            meta = {
                "persona_profile": result.get("persona_profile"),
                "brand": result.get("brand"),
                "product_basic": result.get("product_basic"),
                "product_query": result.get("product_query"),
                "stage_name": result.get("stage_name"),
                "stage_kr": result.get("stage_kr"),
                "objective": result.get("objective"),
                "target_state": result.get("target_state"),
                "style_templates": result.get("style_templates"),
                "selected_event": result.get("selected_event"),
            }
            return summarization, crm_message, meta
        summarization = result.get("summarization")
        crm_message = result.get("crm_message")
        return summarization, crm_message, {}
    if isinstance(result, (list, tuple)) and len(result) >= 2:
        summarization, crm_message = result[0], result[1]
        return summarization, crm_message, {}
    if isinstance(result, str):
        try:
            parsed = json.loads(result)
        except json.JSONDecodeError as exc:
            raise ValueError(f"Unexpected pipeline output: {result}") from exc
        return _extract_pipeline_output(parsed)
    raise ValueError(f"Unexpected pipeline output: {result}")


def _parse_stdout_payload(stdout_text):
    for line in reversed(stdout_text.splitlines()):
        line = line.strip()
        if not line:
            continue
        if not line.startswith("{") or not line.endswith("}"):
            continue
        try:
            return json.loads(line)
        except json.JSONDecodeError:
            continue
    raise ValueError("Pipeline did not return a usable payload.")


def _run_pipeline_via_argv(pipeline_main, row):
    argv = [
        "run_qwen_exaone_pipeline.py",
        "--persona",
        str(row["persona"]),
        "--brand",
        row["brand"],
        "--product",
        row["product"],
        "--stage_index",
        str(row["stage_index"]),
        "--style_index",
        str(row["style_index"]),
        "--is_event",
        "1" if row.get("is_event", False) else "0",
    ]
    if not ENABLE_PIPELINE_CACHE:
        argv.append("--disable_cache")
    buf = StringIO()
    old_argv = sys.argv
    try:
        sys.argv = argv
        with redirect_stdout(buf):
            result = pipeline_main()
    finally:
        sys.argv = old_argv
    if result is not None:
        return result
    return _parse_stdout_payload(buf.getvalue())


def _run_pipeline(pipeline_main, row):
    params = {
        "persona": row["persona"],
        "brand": row["brand"],
        "product": row["product"],
        "stage_index": row["stage_index"],
        "style_index": row["style_index"],
        "is_event": row.get("is_event", False),
    }
    if PIPELINE_MODULE is None:
        _import_pipeline_main()
    if PIPELINE_MODULE is not None:
        if hasattr(PIPELINE_MODULE, "_set_cache_enabled"):
            PIPELINE_MODULE._set_cache_enabled(ENABLE_PIPELINE_CACHE)
        else:
            try:
                PIPELINE_MODULE.CACHE_ENABLED = ENABLE_PIPELINE_CACHE
            except Exception:
                pass
    if PIPELINE_MODULE is not None and hasattr(PIPELINE_MODULE, "_run_pipeline"):
        ctx = _get_pipeline_context()
        args = argparse.Namespace(
            persona=params["persona"],
            brand=params["brand"],
            product=params["product"],
            stage_index=params["stage_index"],
            style_index=params["style_index"],
            is_event=1 if params["is_event"] else 0,
            top_k=PIPELINE_DEFAULTS["top_k"],
            qwen_model=PIPELINE_DEFAULTS["qwen_model"],
            exa_model=PIPELINE_DEFAULTS["exa_model"],
            out_path=None,
            batch_json=None,
            disable_cache=not ENABLE_PIPELINE_CACHE,
        )
        return PIPELINE_MODULE._run_pipeline(
            args,
            data=ctx.get("data"),
            q_generator=ctx.get("q_generator"),
            exa_generator=ctx.get("exa_generator"),
        )

    try:
        sig = inspect.signature(pipeline_main)
    except (TypeError, ValueError):
        sig = None

    if sig is not None and len(sig.parameters) == 0:
        return _run_pipeline_via_argv(pipeline_main, row)

    try:
        return pipeline_main(**params)
    except TypeError:
        pass

    try:
        if sig is not None and len(sig.parameters) == 1:
            return pipeline_main(row)
    except TypeError:
        pass

    ordered = [
        params["persona"],
        params["brand"],
        params["product"],
        params["stage_index"],
        params["style_index"],
        params["is_event"],
    ]
    return pipeline_main(*ordered)


def _run_pipeline_batch(pipeline_main, rows):
    if not ENABLE_BATCH:
        outputs = []
        for row in rows:
            outputs.append(_run_pipeline(pipeline_main, row))
        return outputs
    if PIPELINE_MODULE is None:
        _import_pipeline_main()
    if PIPELINE_MODULE is not None:
        if hasattr(PIPELINE_MODULE, "_set_cache_enabled"):
            PIPELINE_MODULE._set_cache_enabled(ENABLE_PIPELINE_CACHE)
        else:
            try:
                PIPELINE_MODULE.CACHE_ENABLED = ENABLE_PIPELINE_CACHE
            except Exception:
                pass
    if PIPELINE_MODULE is not None and hasattr(PIPELINE_MODULE, "main"):
        payload = []
        for row in rows:
            payload.append({
                "persona": row["persona"],
                "brand": row["brand"],
                "product": row["product"],
                "stage_index": row["stage_index"],
                "style_index": row["style_index"],
                "is_event": row.get("is_event", False),
            })
        with tempfile.NamedTemporaryFile("w", delete=False, suffix=".json", encoding="utf-8") as f:
            json.dump(payload, f, ensure_ascii=False)
            tmp_path = f.name
        argv = [
            "run_qwen_exaone_pipeline.py",
            "--batch_json",
            tmp_path,
            "--qwen_model",
            PIPELINE_DEFAULTS["qwen_model"],
            "--exa_model",
            PIPELINE_DEFAULTS["exa_model"],
            "--top_k",
            str(PIPELINE_DEFAULTS["top_k"]),
        ]
        if not ENABLE_PIPELINE_CACHE:
            argv.append("--disable_cache")
        buf = StringIO()
        old_argv = sys.argv
        try:
            sys.argv = argv
            with redirect_stdout(buf):
                result = pipeline_main()
        finally:
            sys.argv = old_argv
            try:
                os.remove(tmp_path)
            except Exception:
                pass
        if result is not None:
            return result
        return _parse_stdout_payload(buf.getvalue())
    outputs = []
    for row in rows:
        outputs.append(_run_pipeline(pipeline_main, row))
    return outputs


def _collect_candidates(inference_pipeline, row, num_candidates):
    summarization = None
    candidates = []
    summary_mismatch = False
    batch_rows = [row for _ in range(num_candidates)]
    results = _run_pipeline_batch(inference_pipeline, batch_rows)
    if not isinstance(results, list):
        results = [results]
    for attempt, result in enumerate(results, start=1):
        _log(f"  [Attempt {attempt}/{len(results)}] Running pipeline")
        timing = result.get("timing") if isinstance(result, dict) else None
        if timing:
            _log_timing(timing)
        s, message, meta = _extract_pipeline_output(result)
        if summarization is None and s:
            summarization = s
        elif s and summarization and s != summarization:
            summary_mismatch = True
        if isinstance(message, str) and message.strip():
            candidates.append({"text": message.strip(), "meta": meta})
    if summary_mismatch:
        _log("  [Warn] Qwen draft differs across candidates; using the first one.")
    return summarization, candidates


def generate_dpo_data(csv_path, output_path, max_rows=None, num_candidates=4):
    inference_pipeline = _import_pipeline_main()
    records = _load_existing_records(output_path)

    PAIRS_PER_ROW = max(1, num_candidates - 1)
    start_row = len(records) // PAIRS_PER_ROW

    _log(f"Loaded existing records: {len(records)}")
    _log(f"Resuming from CSV row index: {start_row}")

    _log(f"CSV: {csv_path}")
    _log(f"Output: {output_path}")
    _log(f"Candidates per row: {num_candidates}")
    _log(f"Loaded existing records: {len(records)}")

    added = 0
    SAVE_EVERY_N_ROWS = 4
    processed_rows = 0
    for idx, row in enumerate(_load_pairs(csv_path)):
        if idx < start_row or idx < 400 or idx > 500:
          continue
        if max_rows is not None and idx >= max_rows:
            break

        _log(
            "[Row {idx}] persona={persona} brand={brand} product={product} "
            "stage_index={stage_index} style_index={style_index} is_event={is_event}".format(
                idx=idx,
                persona=row["persona"],
                brand=row["brand"],
                product=row["product"],
                stage_index=row["stage_index"],
                style_index=row["style_index"],
                is_event=row.get("is_event", False),
            )
        )

        try:
            summarization, candidates = _collect_candidates(
                inference_pipeline, row, num_candidates
            )
        except Exception as exc:
            _log(f"[Row {idx}] Pipeline error: {exc}")
            continue

        meta_for_prompt = candidates[0].get("meta", {}) if candidates else {}
        prompt_text = _build_prompt_text(meta_for_prompt, summarization)
        if not prompt_text:
            _log(f"[Row {idx}] Empty summarization, skipping")
            continue

        _log(f"[Row {idx}] Raw candidates: {len(candidates)}")
        candidates = _dedupe_candidates(_normalize_candidates(candidates))
        _log(f"[Row {idx}] Deduped candidates: {len(candidates)}")
        if len(candidates) < 2:
            _log(f"[Row {idx}] Not enough candidates, skipping")
            continue

        try:
            best_idx, reason_best, reason_others = _call_gpt(prompt_text, candidates)
            reasons_by_index = {}
            if reason_others:
                for item in reason_others:
                    if not isinstance(item, dict):
                        continue
                    other_idx = item.get("index")
                    if isinstance(other_idx, str) and other_idx.strip().isdigit():
                        other_idx = int(other_idx.strip())
                    if isinstance(other_idx, int):
                        reasons_by_index[other_idx] = item.get("reason") or ""
            if reason_best:
                _log(f"[Row {idx}] Best reason: {reason_best}")
            if reason_others:
                for item in reason_others:
                    if not isinstance(item, dict):
                        continue
                    other_idx = item.get("index")
                    other_reason = item.get("reason") or ""
                    if other_reason:
                        _log(f"[Row {idx}] Other {other_idx}: {other_reason}")
        except Exception as exc:
            _log(f"[Row {idx}] Evaluator error: {exc}")
            continue

        _log(f"[Row {idx}] Best candidate index: {best_idx}")
        best_text = candidates[best_idx]["text"]
        for candidate_idx, candidate in enumerate(candidates):
            if candidate is candidates[best_idx]:
                continue
            rejected_text = candidate["text"]
            if not rejected_text:
                continue
            rejected_reason = reasons_by_index.get(candidate_idx, "")
            records.append(
                {
                    "prompt": prompt_text,
                    "chosen": best_text,
                    "rejected": rejected_text,
                    "best_index": best_idx,
                    "rejected_index": candidate_idx,
                    "reason_best": reason_best,
                    "reason_rejected": rejected_reason,
                }
            )
            added += 1

        # row 1개 처리 완료
        processed_rows += 1

        if processed_rows % SAVE_EVERY_N_ROWS == 0:
          _save_records(output_path, records)
          _log(f"[Checkpoint] Saved after {processed_rows} rows")

    _save_records(output_path, records)
    _log(f"Saved {added} new records to {output_path}")


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--csv_path", default=DEFAULT_CSV)
    parser.add_argument("--output_path", default=DEFAULT_OUTPUT)
    parser.add_argument("--max_rows", type=int, default=None)
    parser.add_argument("--num_candidates", type=int, default=4)
    args, _ = parser.parse_known_args()

    generate_dpo_data(
        args.csv_path,
        args.output_path,
        args.max_rows,
        args.num_candidates,
    )


if __name__ == "__main__":
    main()


Loaded existing records: 303
Resuming from CSV row index: 101
CSV: /content/AmoRe_crm_generator/finetuning/random_persona_campaign.csv
Output: /content/AmoRe_crm_generator/finetuning/finetuning_data_dpo/cycle_01.json
Candidates per row: 4
Loaded existing records: 303
[Row 400] persona=2 brand=설화수 product=탄력크림 EX 75ml stage_index=0 style_index=5 is_event=True
  [Attempt 1/4] Running pipeline
  [Attempt 2/4] Running pipeline
  [Attempt 3/4] Running pipeline
  [Attempt 4/4] Running pipeline
  [Warn] Qwen draft differs across candidates; using the first one.
[Row 400] Raw candidates: 4
[Row 400] Deduped candidates: 4
[Row 400] Best candidate index: 0
[Row 401] persona=3 brand=헤라 product=파워부스팅 스페셜세트 2종 (150ml+110ml) stage_index=3 style_index=3 is_event=True
  [Attempt 1/4] Running pipeline
  [Attempt 2/4] Running pipeline
  [Attempt 3/4] Running pipeline
  [Attempt 4/4] Running pipeline
  [Warn] Qwen draft differs across candidates; using the first one.
[Row 401] Raw candidates: 4
[Row 401]

In [1]:
!pip install huggingface-hub

In [7]:
# Push to HuggingFace
import os
from dotenv import load_dotenv
from huggingface_hub import login, create_repo, upload_folder

load_dotenv("../.env")
login(os.getenv("HUGGINGFACE_API_KEY"))

create_repo(
    repo_id="crm-dpo-dataset",
    repo_type="dataset",
    private=False,
    exist_ok=True
)

upload_folder(
    repo_id="jinn33/crm-dpo-dataset",
    folder_path="./finetuning_data_dpo/crm-dpo-dataset/",
    repo_type="dataset",
    commit_message="Initial Commit"
)

CommitInfo(commit_url='https://huggingface.co/datasets/jinn33/crm-dpo-dataset/commit/b618cb9d24b37a17e151356d65282694f5dbdb38', commit_message='Initial Commit', commit_description='', oid='b618cb9d24b37a17e151356d65282694f5dbdb38', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/jinn33/crm-dpo-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='jinn33/crm-dpo-dataset'), pr_revision=None, pr_num=None)